In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import email.utils
import datetime

In [3]:
dasovich_emails = pd.read_csv('emails/dasovich-j_simple_inbox.csv')
kaminski_emails = pd.read_csv('emails/kaminski-v_simple_inbox.csv')
kean_emails = pd.read_csv('emails/kean-s_simple_inbox.csv')
mann_emails = pd.read_csv('emails/mann-k_simple_inbox.csv')

inboxes = {
    'dasovich': dasovich_emails,
    'kaminski': kaminski_emails,
    'kean': kean_emails,
    'mann': mann_emails
}

# Inbox with the most meeting related emails
most_meeting_related = None
most_meeting_related_count = 0

for name, inbox in inboxes.items():
    meeting_related = inbox[inbox['meeting_related'] == True]
    meeting_related_count = len(meeting_related)
    if meeting_related_count > most_meeting_related_count:
        most_meeting_related = name
        most_meeting_related_count = meeting_related_count

print(f'{most_meeting_related} has the most meeting related emails with {most_meeting_related_count} emails')

dasovich has the most meeting related emails with 2205 emails


In [4]:
dasovich_emails.head()

,file,user,From,To,Subject,Date,content,meeting_related,formatted_email
0,dasovich-j/notes_inbox/5816.,dasovich-j,ginger.dernehl@enron.com,"assad@elektro.com.br, alan.comnes@enron.com, a...",Steve Walton's son,"Fri, 12 Jan 2001 07:28:00 -0800 (PST)",\tCarmen Perez@ENRON_DEVELOPMENT\n\t01/12/2001...,True,From: ginger.dernehl@enron.com\nTo: assad@elek...
1,dasovich-j/notes_inbox/3476.,dasovich-j,rcarroll@bracepatt.com,"ray.alvarez@ei.enron.com, acomnes@enron.com, d...",Fwd: Status- S. 764 (Feinstein bill),"Thu, 10 May 2001 08:58:00 -0700 (PDT)",The attached is an update on the Feinstein bil...,True,From: rcarroll@bracepatt.com\nTo: ray.alvarez@...
2,dasovich-j/notes_inbox/777.,dasovich-j,dhunter@smithandkempton.com,jeff.dasovich@enron.com,RE: AB 1890 Retreat,"Mon, 2 Oct 2000 08:13:00 -0700 (PDT)",Jeff-\nI am always interested! The angle we ar...,True,From: dhunter@smithandkempton.com\nTo: jeff.da...
3,dasovich-j/notes_inbox/2745.,dasovich-j,janel.guerrero@enron.com,"richard.shapiro@enron.com, mark.palmer@enron.c...",campaign leadership conf. call,"Fri, 1 Jun 2001 01:39:00 -0700 (PDT)","Rick,\n\nI'll be on a plane back to Houston th...",True,From: janel.guerrero@enron.com\nTo: richard.sh...
4,dasovich-j/california_crisis__press/17.,dasovich-j,gavin.dillingham@enron.com,"filuntz@aol.com, liz@luntz.com, nicholas.o'day...",WSJ Editorial on Deregulation,"Tue, 29 Aug 2000 05:27:00 -0700 (PDT)",Editorial: Some people call me Dandy. Some peo...,True,From: gavin.dillingham@enron.com\nTo: filuntz@...


In [5]:
for name, inbox in inboxes.items():
    inbox['datetime'] = inbox['Date'].apply(lambda x:
        datetime.datetime.fromtimestamp(
            email.utils.mktime_tz(
                email.utils.parsedate_tz(x)
            )
        ) if email.utils.parsedate_tz(x) else pd.NaT
    )
    inbox['year_week'] = inbox['datetime'].dt.strftime('%Y-%U')

# Inbox and week with the most meeting related emails

most_meeting_related = None
most_meeting_related_week = None
most_meeting_related_week_count = 0

for name, inbox in inboxes.items():
    meeting_related = inbox[inbox['meeting_related'] == True]
    meeting_related_weeks = meeting_related.groupby('year_week').size()
    if meeting_related_weeks.max() > most_meeting_related_week_count:
        most_meeting_related = name
        most_meeting_related_week = meeting_related_weeks.idxmax()
        most_meeting_related_week_count = meeting_related_weeks.max()


print(f'{most_meeting_related} has the most meeting related emails in week {most_meeting_related_week} with {most_meeting_related_week_count} emails')


for name, inbox in inboxes.items():
    print(name)
    meeting_related = inbox[inbox['meeting_related'] == True]
    print(meeting_related['year_week'].value_counts())

mann has the most meeting related emails in week 2001-21 with 113 emails
dasovich
year_week
2001-16    77
2001-20    76
2001-18    60
2001-19    60
2001-27    55
           ..
2000-07     1
2000-11     1
2000-13     1
2000-01     1
1979-52     1
Name: count, Length: 109, dtype: int64
kaminski
year_week
2001-17    73
2000-41    59
2001-16    51
2000-50    44
2000-48    44
           ..
2001-36     2
1999-49     1
1979-52     1
2001-37     1
2001-31     1
Name: count, Length: 106, dtype: int64
kean
year_week
2000-50    69
2001-08    53
2001-06    48
2001-19    43
2001-05    43
           ..
1999-43     1
2001-36     1
2001-38     1
1997-16     1
2001-49     1
Name: count, Length: 145, dtype: int64
mann
year_week
2001-21    113
2001-22    112
2001-19     96
2001-20     91
2001-16     69
          ... 
2001-30      2
2001-40      1
2001-31      1
2000-52      1
2001-36      1
Name: count, Length: 71, dtype: int64


In [7]:

mann_meeting_related_2001_21 = mann_emails[mann_emails['meeting_related'] == True][mann_emails['year_week'] == '2001-21']
print(len(mann_meeting_related_2001_21))

mann_2001_21 = mann_emails[mann_emails['year_week'] == '2001-21']
print(len(mann_2001_21))

mann_meeting_related_2001_21.to_csv('emails/mann_meeting_related_2001_21.csv', index=False)
mann_2001_21.to_csv('emails/mann_2001_21.csv', index=False)

113
154


/var/folders/k6/vyyf41r923779yt71y94ckvh0000gn/T/ipykernel_11415/1801521745.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  mann_meeting_related_2001_21 = mann_emails[mann_emails['meeting_related'] == True][mann_emails['year_week'] == '2001-21']
